In [10]:
import pandas as pd

In [11]:
df = pd.read_json('ds.json')

In [12]:
df.describe()

,externalStatus,internalStatus
count,1222,1222
unique,108,15
top,Gate out,Loaded on Vessel
freq,144,331


In [13]:
df.isnull().sum()

externalStatus    0
internalStatus    0
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222 entries, 0 to 1221
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   externalStatus  1222 non-null   object
 1   internalStatus  1222 non-null   object
dtypes: object(2)
memory usage: 19.2+ KB


In [15]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)


In [16]:
df.describe()

,externalStatus,internalStatus
count,108,108
unique,108,15
top,PORT OUT,Loaded on Vessel
freq,1,49


In [17]:
df['externalStatus'] = df['externalStatus'].str.strip()

In [18]:
# Standardize
df['externalStatus'] = df['externalStatus'].str.lower()
df['internalStatus'] = df['internalStatus'].str.lower()
df['internalStatus'] = df['internalStatus'].str.strip()

# Convert categorical columns to categorical data type
df['externalStatus'] = df['externalStatus'].astype('category')
df['internalStatus'] = df['internalStatus'].astype('category')


In [19]:
df.head()

,externalStatus,internalStatus
0,port out,port out
1,terminal in,inbound terminal
2,port in,port in
3,vessel departure from first pol (vessel name :...,departure
4,vessel arrival at final pod (vessel name : tia...,arrival


In [20]:
df.describe()

,externalStatus,internalStatus
count,108,108
unique,105,15
top,loaded on board,loaded on vessel
freq,2,49


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 1016
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   externalStatus  108 non-null    category
 1   internalStatus  108 non-null    category
dtypes: category(2)
memory usage: 6.6 KB


In [22]:
#csv convertion
df.to_csv('cleaned_ds.csv', index=False)


In [23]:
df.head()

,externalStatus,internalStatus
0,port out,port out
1,terminal in,inbound terminal
2,port in,port in
3,vessel departure from first pol (vessel name :...,departure
4,vessel arrival at final pod (vessel name : tia...,arrival


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 0 to 1016
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   externalStatus  108 non-null    category
 1   internalStatus  108 non-null    category
dtypes: category(2)
memory usage: 6.6 KB


In [25]:
df=pd.read_csv('cleaned_ds.csv')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   externalStatus  108 non-null    object
 1   internalStatus  108 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [27]:

from sklearn.preprocessing import LabelEncoder
ext_enc = LabelEncoder()
int_enc = LabelEncoder()

df['ext_encoded'] = ext_enc.fit_transform(df['externalStatus'])
df['int_encoded'] = int_enc.fit_transform(df['internalStatus'])

df.to_csv('cleaned_ds.csv', index=False)

In [28]:
df

,externalStatus,internalStatus,ext_encoded,int_encoded
0,port out,port out,89,13
1,terminal in,inbound terminal,91,7
2,port in,port in,88,12
3,vessel departure from first pol (vessel name :...,departure,104,1
4,vessel arrival at final pod (vessel name : tia...,arrival,101,0
...,...,...,...,...
103,'marina one 001s' arrival at port of dischargi...,arrival,0,0
104,unloaded from 'marina one 001s' at port of dis...,unloaded on vessel,95,14
105,load on mogens maersk / 246w,loaded on vessel,59,8
106,load on msc alexandra / 248w,loaded on vessel,60,8


In [29]:
import tensorflow as tf

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(df['internalStatus'].unique()), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [35]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X = df['ext_encoded'].values.reshape(-1, 1)

In [38]:
y = df['int_encoded'].values

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.2950 - loss: 2.3871 - val_accuracy: 0.3333 - val_loss: 2.5684
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1749 - loss: 2.6791 - val_accuracy: 0.0556 - val_loss: 2.5847
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.3701 - loss: 2.0725 - val_accuracy: 0.0556 - val_loss: 2.5901
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.1697 - loss: 2.5991 - val_accuracy: 0.3333 - val_loss: 2.5720
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2800 - loss: 2.6725 - val_accuracy: 0.3333 - val_loss: 2.5632
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3297 - loss: 2.2860 - val_accuracy: 0.3333 - val_loss: 2.5679
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.3701 - loss: 2.1695 - val_accuracy: 0.3333 - val_loss: 2.5685
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3949 - loss: 2.0396 - val_accuracy: 0.3333 - val_loss: 2.5598

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [43]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step


In [44]:
precision = precision_score(y_test, y_pred_classes, average='weighted', zero_division=0)

In [45]:
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_classes, average='weighted', zero_division=0)

In [46]:
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

Accuracy: 0.50
Precision: 0.25
Recall: 0.50


In [47]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)), 
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(128, activation='relu'),  
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(len(df['internalStatus'].unique()), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [51]:
model.save("model.keras")

In [52]:
import requests

# Define the external status data
your_external_data = {
    "external_status": "Your external status data here"
}

# Send a POST request to the FastAPI server
response = requests.post("http://127.0.0.1:8000/predict/", json=your_external_data)

# Check if the request was successful
if response.status_code == 200:
    # Print the predicted internal status
    print("Predicted internal status:", response.json())
else:
    print("Error:", response.text)


Error: Internal Server Error


In [58]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming you have already prepared your data and defined df

# Define your model architecture
model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(df['internalStatus'].unique()), activation='softmax')
])

# Compile the model
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Prepare data for training
X = df['ext_encoded'].values.reshape(-1, 1)
y = df['int_encoded'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model2.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model for later use
model2.save("your_model2.keras")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.3715 - loss: 2.3022 - val_accuracy: 0.5000 - val_loss: 2.2118
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3677 - loss: 2.2575 - val_accuracy: 0.5000 - val_loss: 2.2051
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4165 - loss: 2.2232 - val_accuracy: 0.5000 - val_loss: 2.2093
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3755 - loss: 2.1238 - val_accuracy: 0.5000 - val_loss: 2.2170
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4184 - loss: 2.1204 - val_accuracy: 0.5000 - val_loss: 2.2273
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3639 - loss: 2.1837 - val_accuracy: 0.5000 - val_loss: 2.2426
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3327 - loss: 2.2740 - val_accuracy: 0.5000 - val_loss: 2.2500
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3755 - loss: 2.1816 - val_accuracy: 0.5000 - val_loss: 2.2255


In [68]:
import requests

# Define the URL of the endpoint
url = "http://127.0.0.1:8000/predict/"

# Define the external status data
external_status_data = {
    "external_status": "port out"
}

# Send the POST request
response = requests.post(url, json=external_status_data)

# Check if the request was successful
if response.status_code == 200:
    # Print the response data
    print(response.json())
else:
    # Print an error message
    print("Error:", response.text)


Error: Internal Server Error
